In [53]:
import http.client, urllib.parse, json
import pandas as pd
import os
#from util_html import *
import requests
import re
from bs4 import BeautifulSoup
import html5lib

In [2]:
def guardian_query(keyword, from_date, to_date, page='1', page_size='200'):
    '''
    Get query content
    example usage: guardian_query(keyword='abortion',from_date='2022-06-24',to_date='2022-07-24')
    '''
    
    api_key = 'ca17bcab-db02-44a8-8a32-dd408dfbffcd'
    
    conn = http.client.HTTPConnection('content.guardianapis.com')
    params = ('from-date=' + str(from_date) + '&to-date=' + str(to_date) + 
              '&page=' + str(page) + '&page-size=' + str(page_size) + '&q=' + str(keyword) +
              '&api-key=' + str(api_key))
    
    conn.request('GET', '/search?{}'.format(params))

    res = conn.getresponse()
    data = res.read()
    query_content=(data.decode('utf-8'))
    query = json.loads(query_content)
    
    return query

In [3]:
guardian_query(keyword='abortion',from_date='2022-06-24',to_date='2022-09-24')

{'response': {'status': 'ok',
  'userTier': 'developer',
  'total': 294,
  'startIndex': 1,
  'pageSize': 200,
  'currentPage': 1,
  'pages': 2,
  'orderBy': 'relevance',
  'results': [{'id': 'global-development/2022/jul/22/european-countries-pressurise-uk-over-removal-of-abortion-commitments-liz-truss',
    'type': 'article',
    'sectionId': 'global-development',
    'sectionName': 'Global development',
    'webPublicationDate': '2022-07-22T12:02:18Z',
    'webTitle': 'UK under international pressure over deletion of abortion commitments',
    'webUrl': 'https://www.theguardian.com/global-development/2022/jul/22/european-countries-pressurise-uk-over-removal-of-abortion-commitments-liz-truss',
    'apiUrl': 'https://content.guardianapis.com/global-development/2022/jul/22/european-countries-pressurise-uk-over-removal-of-abortion-commitments-liz-truss',
    'isHosted': False,
    'pillarId': 'pillar/news',
    'pillarName': 'News'},
   {'id': 'media/2022/jul/09/bbc-bitesize-gave-platfor

In [254]:
query_abortion_1 = guardian_query(keyword='abortion',from_date='2022-06-24',to_date='2022-09-24', page='1')
query_abortion_2 = guardian_query(keyword='abortion',from_date='2022-06-24',to_date='2022-09-24', page='2')
query_abortion_3 = guardian_query(keyword='abortion',from_date='2022-06-24',to_date='2022-09-24', page='3')
metadata_abortion_1 = get_guardian_metadata(query_abortion_1)
metadata_abortion_2 = get_guardian_metadata(query_abortion_2)
metadata_abortion_3 = get_guardian_metadata(query_abortion_3)
metadata_abortion = metadata_abortion_1 + metadata_abortion_2 + metadata_abortion_3

In [283]:
query = [query_abortion_1,query_abortion_2,query_abortion_3]
len(query)

3

In [288]:
def get_guardian_metadata(query):
    section = []
    date = []
    title = []
    urls = []
    for i in range(len(query)):
        articles = (query[i]['response'])['results']
        max_link = 1000
        for j, article in enumerate(articles):
            if j < max_link:
                section.append(article['sectionName'])
                date.append(article['webPublicationDate'])
                title.append(article['webTitle'])
                urls.append(article['webUrl'])
    return section, date, title, urls

In [240]:
def get_guardian_metadata(query):
    section = []
    date = []
    title = []
    urls = []
    articles = (query['response'])['results']
    max_link = 1000
    for j, article in enumerate(articles):
        if j < max_link:
            section.append(article['sectionName'])
            date.append(article['webPublicationDate'])
            title.append(article['webTitle'])
            urls.append(article['webUrl'])
    return section, date, title, urls

In [5]:
'''
def get_guardian_urls(keyword, from_date, to_date, page_range):
    '''
    #Get the urls from the query json
    #example usage: urls = get_guardian_urls(keyword='abortion',from_date='2022-06-24',to_date='2022-07-24', page_range=2)
    '''
    urls = []
    for i in (1,page_range):
        query = guardian_query(keyword=keyword, from_date=from_date, to_date=to_date, page=i)
        articles = (query['response'])['results']
        max_link = 1000
        for j, article in enumerate(articles):
            if j < max_link:
                urls.append(article['webUrl'])
    return urls
'''

In [6]:
#urls = get_guardian_urls(keyword='abortion',from_date='2022-06-24',to_date='2022-07-24', page_range=2)

In [8]:
get_guardian_content(urls[0])

'Exclusive: open letter sent to Liz Truss after removal of commitment to repeal laws threatening women and girls’ rights from global pactThe UK government is coming under growing pressure from European countries and human rights groups to explain why commitments to abortion and sexual health rights have been removed from an official statement on gender equality.Norway and Denmark have approached the Foreign, Commonwealth and Development Office (FCDO) “to protest against the substantive changes” that were made to a paper that resulted from a UK-hosted conference on freedom of religion and belief, opened by Liz Truss earlier this month, the Guardian has learned.More than 20 countries, including those now complaining, had signed the original text, which included a commitment to the repeal of any laws that “allow harmful practices, or restrict women’s and girls’ … sexual and reproductive health and rights, bodily autonomy.”But those phrases were removed from a later version of the internat

In [116]:
def metadata_train_test_split(metadata, n = 20):
    '''
    Split the train/test with n% of test data
    '''
    df = pd.DataFrame(metadata).T.rename(columns={0:'section', 1:'time', 2:'title', 3:'url'})
    rdf = df.sample(frac=1, random_state=1) # reproducibility
    
    test = int(len(df)*(n/100))
    train = len(df)
    
    test_index = rdf.index[0:test]
    train_index = rdf.index[test:train]
    
    test_metadata = rdf.head(test)
    train_metadata = rdf.tail(train-test)
    
    return train_metadata, test_metadata, train_index, test_index

In [108]:
out_dir = '/home/arimo/Desktop/Studily/LD/labs/Lab1_Crawling/code/out'

def create_folders_if_not_exist(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)

def create_folders(dir):
    '''
    Creates folders in the directory if they do not exist
    Folder Structure:
    ├── * input directory
    │   ├── language_1
    │   │   ├── train
    │   │   ├── test
    │   ├── language_2
    │   │   ├── train
    │   │   ├── test
    '''
    create_folders_if_not_exist(dir)
    
    lang1_dir = os.path.join(dir, r'language_1')
    create_folders_if_not_exist(lang1_dir)
    lang1_train_dir = os.path.join(lang1_dir, r'train')
    create_folders_if_not_exist(lang1_train_dir)
    lang1_test_dir = os.path.join(lang1_dir, r'test')
    create_folders_if_not_exist(lang1_test_dir)
        
    lang2_dir = os.path.join(dir, r'language_2')
    create_folders_if_not_exist(lang2_dir)
    lang2_train_dir = os.path.join(lang2_dir, r'train')
    create_folders_if_not_exist(lang2_train_dir)
    lang2_test_dir = os.path.join(lang2_dir, r'test')
    create_folders_if_not_exist(lang2_test_dir)

    return lang1_train_dir, lang1_test_dir, lang2_train_dir, lang2_test_dir

dirs = create_folders(out_dir)

In [292]:
dirs

('/home/arimo/Desktop/Studily/LD/labs/Lab1_Crawling/code/out/language_1/train',
 '/home/arimo/Desktop/Studily/LD/labs/Lab1_Crawling/code/out/language_1/test',
 '/home/arimo/Desktop/Studily/LD/labs/Lab1_Crawling/code/out/language_2/train',
 '/home/arimo/Desktop/Studily/LD/labs/Lab1_Crawling/code/out/language_2/test')

In [129]:
def write_guardian_metadata(dirs, outfile_keywords, splittedmetadata):
    trainfile = dirs[0] + '/' + outfile_keywords + "_train_metadata.csv"
    testfile = dirs[1] + '/' + outfile_keywords + "_test_metadata.csv"
    splittedmetadata[0].to_csv(trainfile)
    splittedmetadata[1].to_csv(testfile)

In [117]:
splittedmetadata = metadata_train_test_split(metadata_abortion)

In [130]:
write_guardian_metadata(dirs, 'abortion', splittedmetadata)

In [135]:
splittedmetadata[0]

,section,time,title,url
108,Opinion,2022-07-01T10:19:13Z,I read the 1973 Roe v Wade ruling to see what ...,https://www.theguardian.com/commentisfree/2022...
107,Opinion,2022-06-26T16:40:34Z,The Guardian view on overturning Roe v Wade: a...,https://www.theguardian.com/commentisfree/2022...
189,US news,2022-07-13T13:02:09Z,Republican Josh Hawley accused of transphobia ...,https://www.theguardian.com/us-news/2022/jul/1...
14,World news,2022-06-24T18:01:28Z,Abortion deserts: America’s new geography of a...,https://www.theguardian.com/world/2022/jun/24/...
56,Opinion,2022-07-09T13:00:06Z,Restoring US abortion access won’t be easy – b...,https://www.theguardian.com/commentisfree/2022...
...,...,...,...,...
133,World news,2022-06-25T07:00:08Z,How the Christian right took over the judiciar...,https://www.theguardian.com/world/2022/jun/25/...
137,Technology,2022-06-29T10:45:37Z,TechScape: Can US women trust big tech with th...,https://www.theguardian.com/technology/2022/ju...
72,Australia news,2022-06-30T23:33:09Z,South Australian Liberal leader and state MPs ...,https://www.theguardian.com/australia-news/202...
140,US news,2022-07-09T08:00:00Z,Biden in crisis mode as specter of one-term Ca...,https://www.theguardian.com/us-news/2022/jul/0...


In [7]:
def get_guardian_content(url):
    '''
    Get the text from url link
    '''
    soup = BeautifulSoup(requests.get(url).text,"html5lib")
    text = ""
    for item in soup.select("p",{"class":"dcr-1kas69x"}): # Guardian body text class
        text += item.text.strip()
    return text

In [190]:
#test = splittedmetadata[1]
(splittedmetadata[1].reset_index())['title'][0].replace('/','_')

'Leak of Dobbs opinion brought deluge of online searches for abortion pills, study finds'

In [193]:
def write_guardian_output(splitted_metadata, keyword, dirs):
    # train
    train = splittedmetadata[0].reset_index()
    for i in range(len(train)):
        title = train['title'][i]
        url = train['url'][i]
        text = get_guardian_content(url)
        dir = dirs[0]
        filename = keyword + "_" + str(title).replace('/','_') + ".txt"
        with open(dir + "/" + filename, "w", encoding = "utf-8") as f:
            f.write(text)
    
    # test
    test = splittedmetadata[1].reset_index()
    for j in range(len(test)):
        title = test['title'][j]
        url = test['url'][j]
        text = get_guardian_content(url)
        dir = dirs[1]
        filename = keyword + "_" + str(title).replace('/','_') + ".txt"
        with open(dir + "/" + filename, "w", encoding = "utf-8") as f:
            f.write(text)

In [194]:
write_guardian_output(splittedmetadata, 'abortion', dirs)

In [ ]:
for i, link in enumerate(search_results):    
    found_url = domain + link["href"]
    print(i, found_url)
    
    # Extract text and add the url as first line
    text = found_url + '\n'+ url_to_string(found_url) 
    
    # Save in file
    dir = "../results/guardian_search_results/"
    filename = keyword + "_" + str(i) + ".txt"
    with open(dir + filename, "w", encoding = "utf-8") as f:
        f.write(text)